# Lab - Cross Validation and Bootstrap

In [14]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize ,
poly)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [2]:
from functools import partial
from sklearn.model_selection import \
(cross_validate ,
KFold ,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

## Validation Set Approach

We explore the use of the validation set approach. 

We use train_test_split() to split the data into training and validation sets. Then we can fit a model using only the observations from the training set.

In [3]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

In [4]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

We now use the predict() method of results on the validation data. We also calculate the validation MSE for our model. It turns out our Test MSE is 23.6

In [5]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred) ** 2)

23.61661706966988

Let's generalize calculating the Test MSE with a function and let's estimate it for linear, quadratic and cubic fits. 

In [18]:
def evalMSE(terms, response, train, test): 
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)

    return np.mean((test_pred - y_test ) ** 2)

In [19]:
MSE = np.zeros(3)

for idx, degree in enumerate(range(1,4)): 
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)

MSE

TypeError: 'PolynomialFeatures' object is not callable

## Cross-Validation

The simplest way to cross validate in python is using sklearn, which has a different interface than statsmodels. 

cross_validate takes an optional parameter cv that indicates the K-fold for K-fold cross validation

In [20]:
model = LinearRegression()

X, Y = Auto[['horsepower']], Auto['mpg']
cv_results = cross_validate(model, X, Y, cv=10) # specifying cv does a k-fold cross validation
# the fact that we provided the shape of the dataset means that we will perform Leave-One-Out CV (LOOCV)
cv_err = np.mean(cv_results['test_score'])
cv_err

0.19549935010445704

We repeat this for various degrees of the polynomial

In [27]:
cv_error = np.zeros(5)
Y = Auto['mpg']
cv = KFold(n_splits=10, shuffle=True, random_state=1)
model = LinearRegression()

for idx, degree in enumerate(range(1,6)): 
    col = f'degree_{degree}'
    poly = PolynomialFeatures(degree=degree)
    poly_features = poly.fit_transform(Auto[['horsepower']])

    cv_results = cross_validate(model, 
                                poly_features, 
                                Y, 
                                cv=cv,
                                scoring='neg_mean_squared_error')
    cv_error[idx] = np.mean(cv_results['test_score'])

cv_error


array([-24.09767573, -19.17888986, -19.21385952, -19.21280702,
       -18.75799181])

In [24]:
print(cv_error)

[-27.43993365 -21.23584006 -21.33660618 -21.35388699 -20.90567054]


# Bootstrap

In [28]:
Portfolio = load_data('Portfolio')
def alpha_func(D,idx): 
    cov_ = np.cov(D[['X', 'Y']].loc[idx], rowvar=False)
    # this returns an estimate for alpha
    return ((cov_[1,1] - cov_[0,1]) / (cov_[0,0] + cov_[1,1,] - 2 * cov_[0,1]))

In [31]:
alpha_func(Portfolio, range(100))

# now we choose the indices randomly
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100,100,replace=True))

0.6074452469619004

We generalize the function. boot_SE finds the standard error estimate for a parameter (in this case $\alpha$). The function runs the bootstrap $B$ times selecting $n$ elements from the sample

In [36]:
# we generalize with a function
def boot_SE(func,D,n=None,B=1000,seed=0): 
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]

    for _ in range(B): 
        idx = rng.choice(D.index, n,replace=True)
        value = func(D,idx)
        first_ += value
        second_ += value ** 2

    return np.sqrt(second_ / B - (first_ / B) ** 2)

In [38]:
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
alpha_SE

0.09118176521277699

The bootstrap can be used to assess the variability (SE) of the coefficients and predictions from a statistical learning method. 

Let's estimate the variability in the coefficients $\beta_0,\beta_1$, for the linear regression that estimates mpg from horsepower in the Auto dataset. We then compare the results to the traditional formulas for $SE(\hat{\beta_0}), SE(\hat{\beta_1})$. 

To use our boot_SE function we must create a function that takes a dataframe D and indices idx as its only arguments. But here we want to boostrap a specific regression model, specified by a model formula and data. How to do this? We define a generic function boot_OLS() for bootstrapping a regression model that takes a formula to define the corresponding regression. We use .clone() to create a copy of the formula which can be refit to the new dataframe. Any derived features, e.g. the ones derived by poly(), will be refit on the resampled dataframe.

In [47]:
def boot_OLS(model_matrix, response,D,idx): 
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)

    return sm.OLS(Y_,X_).fit().params

But this cannot be feeded into boot_SE. We need to freeze the first two arguments since they will stay the same. We can use partial() from the functools module, which takes a function and freezes some of its parameters (starting from the left) returning a new function

In [50]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)
hp_se
# this means the SE for the intercept (beta0) is 0.84 
#                SE for horsepower (beta1) is 0.007

intercept     0.848807
horsepower    0.007352
dtype: float64

In [55]:
# we compare to the standard formulas for SE
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
hp_model.fit(Auto[['horsepower']], Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64


The estimates differ because the standard formulas assume that $x_i$ are fixed and the variation comes only from the residuals. They also depend on $\sigma^2$ which is an estimate and relies on the assumption that the model is correct. The bootstrap does not make such assumptions so it is more correct